In [ ]:
%matplotlib widget
from spectraclass.data.base import DataManager
from matplotlib.image import AxesImage
import numpy as np
from cartopy.mpl.geoaxes import GeoAxes
from spectraclass.xext.xgeo import XGeo
from typing import List, Union, Tuple, Optional, Dict, Callable
from spectraclass.data.spatial.tile.manager import TileManager, tm
import matplotlib.pyplot as plt
import ipywidgets as ipw
import time, xarray as xa

t0 = time.time()
dm: DataManager = DataManager.initialize( "mgr.1", 'aviris' )
dm.modal.cache_dir = "/adapt/nobackup/projects/ilab/cache"
dm.modal.data_dir = "/css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data/"
dm.modal.ext = "_img"
dm.proc_type = "cpu"
TileManager.block_size = 250
nimages = len( dm.modal.image_names )
band_index = 160
print( f"found {nimages} images "  )
print( f"Completed init in {time.time()-t0} sec")

def get_band_data( ) -> np.ndarray:
    dm.modal.set_current_image( 0 )
    data_array: xa.DataArray = tm().tile.data
    band_array: np.ndarray = data_array[band_index].values.squeeze()
    nodata = data_array.attrs.get('_FillValue')
    band_array[band_array == nodata] = np.nan
    return band_array

In [ ]:
ax0 = plt.axes( )
file_selector = dm.modal.file_selector
band_plot: AxesImage = ax0.imshow( get_band_data(), cmap="jet" )

def on_image_change( event: Dict ):
    dm.modal.set_current_image( file_selector.index )
    band_plot.set_data( get_band_data( ) )

dm.modal.set_file_selection_observer( on_image_change )
ipw.HBox( [ band_plot.figure.canvas, file_selector ], layout=ipw.Layout(flex='1 1 auto') )